In [1]:
import numpy as np
import matplotlib.pyplot as plt
import os
import time

In [2]:
Amax = 9 #Generated trajectories with an Amax of 9
random_walks = np.loadtxt("rand_traj/good_random_traj_"+str(Amax)+".txt", dtype=int,skiprows=1)

Our weights depend on where you are, where you're going to, how many copies of the multiplet you have, and how far along in the walk you are (number of gluons added --> a)

In [3]:
possible_a = np.array(range(1,Amax)) #not interested in a=0 since that's constrained to always go to (1,1)
possible_p = np.array(range(np.amax(random_walks[:,1])+1))
possible_q = np.array(range(np.amax(random_walks[:,2])+1))
possible_n = np.array(range(np.amax(random_walks[:,3])+1))

In [4]:
#bar = progressbar.ProgressBar(maxval=1323,widgets=[progressbar.Bar('=', '[', ']'), ' ', progressbar.Percentage()])
#bar.start()
#s = time.time()


total_combinations = possible_a.size * possible_p.size * possible_q.size * possible_n.size
error_array = np.zeros((total_combinations, 5))

"""for a in possible_a:
    error_array[num_iter,0] = a
    
    for p in possible_p:
       
        error_array[num_iter,1] = p
        
        for q in possible_q:
            
            error_array[num_iter,2] = q
        
            for n in possible_n:
                
                error_array[num_iter,3] = n
                
                matching_indicies = []
                mask = (random_walks[:,0] == a) & (random_walks[:,1] == p) & (random_walks[:,2] == q) & (random_walks[:,3] == n)
                for i in range(mask.size):
                    if mask[i] == True:
                        matching_indicies.append(i)
                if (len(matching_indicies) == 0):
                    error_array[num_iter,4] = -1
                    
                num_iter += 1
                #bar.update(num_iter)
"""
#e=time.time()
#bar.finish()
#print(str(np.round(e-s,2))+" seconds")

'for a in possible_a:\n    error_array[num_iter,0] = a\n    \n    for p in possible_p:\n       \n        error_array[num_iter,1] = p\n        \n        for q in possible_q:\n            \n            error_array[num_iter,2] = q\n        \n            for n in possible_n:\n                \n                error_array[num_iter,3] = n\n                \n                matching_indicies = []\n                mask = (random_walks[:,0] == a) & (random_walks[:,1] == p) & (random_walks[:,2] == q) & (random_walks[:,3] == n)\n                for i in range(mask.size):\n                    if mask[i] == True:\n                        matching_indicies.append(i)\n                if (len(matching_indicies) == 0):\n                    error_array[num_iter,4] = -1\n                    \n                num_iter += 1\n                #bar.update(num_iter)\n'

In [29]:
start = time.time()

dp = [2, 1, 0, -1, -2, 1, -1]
dq = [-1, 1, 0, 2, 1, -2, -1]

def get_good_pq(p_given,q_given,delta_p,delta_q,maxval):
    good_pq = []
    for i in range(len(dp)):
        p_new = p_given - delta_p[i];
        q_new = q_given - delta_q[i];
        
        if ((p_new >= 0) & (q_new >= 0) & (p_new <= maxval) & (q_new <= maxval)):
            good_pq.append((p_new,q_new))
    return good_pq

num_iter = 0

a = 1
p = 1
q = 1
n = 1

error_array[num_iter,4] = 0

#this block of code finds where this combination of variables appears in our random walks
matching_indicies = []
mask = (random_walks[:,0] == a) & (random_walks[:,1] == p) & (random_walks[:,2] == q) & (random_walks[:,3] == n)
for i in range(mask.size): 
    if mask[i] == True:
        matching_indicies.append(i)
    
if (len(matching_indicies) == 0): #if there are no instances of this combination, make a 'note' of that to throw
    error_array[num_iter,4] = -1  #it out later
      

goodpq = get_good_pq(p,q,dp,dq,np.amax(possible_p)) #grab the possible multiplets we can jumpt too
number_events = [0]*len(goodpq)                     #a place to store how many times that jump is made
random_events_weights_dict = dict(zip(goodpq,number_events))#dictionary to treat these as one object

for k in matching_indicies:     #find how many times each "jump" was made
    p_next, q_next = random_walks[k+1,1], random_walks[k+1,2]
    random_events_weights_dict[(p_next,q_next)] += 1./len(matching_indicies)

print(random_events_weights_dict)


os.system("./calc_test_weight "+str(a)+" "+str(p)+" "+str(q)+" "+str(n)+" "+str(Amax))

end = time.time()
print(str(np.round(end-start,2)) + " seconds")

{(0, 0): 0.03402259364471125, (1, 1): 0.37101142894895744, (3, 0): 0.15954284204268385, (0, 3): 0.14877452599591015, (2, 2): 0.28664860936779174}
0.18 seconds
